In [243]:
import aocd
import numpy as np

Calculate the result of multiplying the secret number by 64.
Then, mix this result into the secret number. Finally, prune the secret number.

Calculate the result of dividing the secret number by 32.
Round the result down to the nearest integer. 
Then, mix this result into the secret number. Finally, prune the secret number.

Calculate the result of multiplying the secret number by 2048.
Then, mix this result into the secret number. Finally, prune the secret number.

expected 10 first results for `123`:

```
15887950
16495136
527345
704524
1553684
12683156
11100544
12249484
7753432
5908254
```

In [244]:
def mix(n1, n2):
    return n1 ^ n2

def prune(s):
    return s%16777216
    

In [270]:
def next_number(number):
    number = mix(number*64, number)
    number = prune(number)
    number = mix((number//32), number)
    number = prune(number)
    number = mix(number*2048, number)
    number = prune(number)
    return number

In [271]:
sn = 123
for i in range(10):
    sn = next_number(sn)
    print(sn)

15887950
16495136
527345
704524
1553684
12683156
11100544
12249484
7753432
5908254


In [272]:
def calculate_nth_number(secret_number, n = 2000):
    for i in range(n):
        secret_number = next_number(secret_number)
    return secret_number

In [273]:
sum([calculate_nth_number(x) for x in [1, 10, 100, 2024]])

37327623

In [285]:
numbers = aocd.get_data(day = 22, year = 2024)

In [286]:
numbers = [int(x) for x in numbers.split("\n")]

In [287]:
%%time
sum([calculate_nth_number(x) for x in numbers])

CPU times: user 1.53 s, sys: 0 ns, total: 1.53 s
Wall time: 1.53 s


14691757043

In [289]:
numbers

[2681088,
 4082700,
 13359711,
 14520022,
 3226591,
 12421466,
 15724966,
 12648620,
 6501396,
 3516393,
 4904151,
 8274733,
 12728777,
 9737373,
 1945467,
 3087983,
 12201256,
 11259446,
 13918095,
 2974825,
 6715634,
 12773455,
 14237722,
 8371351,
 8151123,
 13481080,
 10708427,
 2165998,
 11729576,
 13960627,
 9860559,
 6200486,
 4760144,
 12195250,
 7083714,
 15502990,
 9731347,
 12080590,
 10299037,
 1427243,
 797789,
 8800667,
 4383480,
 4504622,
 1741195,
 11855636,
 8996374,
 13851815,
 5590427,
 3601174,
 15038078,
 1599662,
 5544963,
 12947626,
 11957226,
 6730346,
 16659709,
 7635174,
 6917799,
 6831574,
 16372839,
 12695779,
 12241266,
 7454445,
 14425705,
 7433471,
 14441747,
 662443,
 13574723,
 11241000,
 13188069,
 13123267,
 2581795,
 11585640,
 15265181,
 11890137,
 13621915,
 7895789,
 13526138,
 3379361,
 11183393,
 7795483,
 16562212,
 11752089,
 198467,
 13723374,
 15919930,
 4000908,
 5778532,
 2384070,
 993966,
 1510353,
 4845213,
 10826071,
 3091629,
 5672895,

In [293]:
%%time
#faster:
sum(calculate_nth_number(np.array(numbers)))

CPU times: user 53.2 ms, sys: 2.91 ms, total: 56.1 ms
Wall time: 55.3 ms


np.int64(14691757043)

In [294]:
#for part two Im going to want a dataframe :/
import pandas as pd

In [295]:
def track_sequences(numbers, cycles = 2000):
    diffs = np.zeros_like(numbers)
    prices = np.zeros_like(numbers)
    for cycle in range(cycles):
        nannervalues = numbers % 10
        next_numbers = calculate_nth_number(numbers, 1)
        next_nannervalues = next_numbers % 10
        diffs = np.vstack([diffs, next_nannervalues-nannervalues])
        prices = np.vstack([prices, nannervalues])
        numbers = next_numbers
    return diffs, prices

In [345]:
test_numbers = np.array([1,2,3,2024])
test_numbers2 = np.array([123])

NameError: name 'real_numbers' is not defined

In [ ]:
d, p = track_sequences(np.array(numbers))

In [342]:
p.shape

(2001, 4)

In [343]:
from collections import defaultdict

In [344]:
nannerdict = {}

In [335]:
for i in range(4, len(d)):
    seqs = d[i-4:i, :]
    for n, seq in enumerate(seqs.T):
        seq = tuple(seq)
        if seq not in nannerdict:
            nannerdict[seq] = {i: p[i,n]}
        elif n not in nannerdict[seq]:
            nannerdict[seq].update({n:p[i,n]})

In [336]:
highest_seq = None
highest_val = 0
for seq in nannerdict:
    val = 0
    vals = nannerdict[seq]
    for v in vals:
        val+=vals[v]
    if val>highest_val:
        highest_val = val
        highest_seq=seq

In [337]:
highest_seq

(np.int64(5), np.int64(-5), np.int64(-3), np.int64(8))

In [338]:
nannerdict[highest_seq]

{198: np.int64(9), 2: np.int64(9), 1: np.int64(9)}

In [339]:
highest_val

np.int64(27)

In [314]:
for seq in seqs.T:
    print(tuple(seq))

(np.int64(0), np.int64(2), np.int64(0), np.int64(6))
(np.int64(0), np.int64(4), np.int64(-4), np.int64(2))
(np.int64(0), np.int64(0), np.int64(0), np.int64(-2))
(np.int64(0), np.int64(5), np.int64(-4), np.int64(-2))


In [309]:
d[0:4, :]

array([[ 0,  0,  0,  0],
       [ 2,  4,  0,  5],
       [ 0, -4,  0, -4],
       [ 6,  2, -2, -2]])

In [298]:
for col in range(p.shape[-1]):
    

array([[ 0,  0,  0,  0],
       [ 2,  4,  0,  5],
       [ 0, -4,  0, -4],
       ...,
       [ 3,  1,  2, -6],
       [-1,  0,  5,  7],
       [ 7, -3, -4, -4]])

In [229]:
from scipy.signal import convolve2d, correlate2d

Spent a bunch of time figuring out why my convolution idea didnt work. turns out I need correlate to not flip the kernel...

In [240]:
# Input sequence (vertical array)
sequence = np.array([[-2],
                     [ 1],
                     [-1],
                     [ 3]])

# Use the sequence directly as the kernel
kernel = sequence.copy()

# Example test array (this should match the sequence exactly)
test_array = np.array([[1],
                       [-2],
                       [ 1],
                       [-1],
                       [ 3],
                       [1]])

# Perform 2D convolution
# result = convolve2d(test_array, np.flip(kernel), mode = "valid")
result = correlate2d(d, kernel, mode="valid")

# Compute the match value (sum of squared elements in the sequence)
match_value = np.sum(sequence*sequence)

# Check for matches
matches = (result == match_value).astype(int)



In [241]:
np.argwhere(matches)

array([[  15,    2],
       [  16,    1],
       [  29,    3],
       [  37,    3],
       [  41,    3],
       [  50,    3],
       [  60,    2],
       [  67,    1],
       [  74,    0],
       [  84,    0],
       [  87,    2],
       [  96,    3],
       [ 122,    1],
       [ 137,    1],
       [ 149,    0],
       [ 176,    0],
       [ 185,    2],
       [ 228,    3],
       [ 254,    0],
       [ 283,    1],
       [ 288,    1],
       [ 306,    3],
       [ 319,    2],
       [ 326,    2],
       [ 351,    3],
       [ 374,    2],
       [ 380,    1],
       [ 418,    3],
       [ 420,    3],
       [ 424,    1],
       [ 441,    3],
       [ 442,    0],
       [ 452,    3],
       [ 475,    2],
       [ 485,    0],
       [ 514,    2],
       [ 518,    2],
       [ 528,    2],
       [ 532,    0],
       [ 560,    3],
       [ 587,    0],
       [ 590,    1],
       [ 612,    1],
       [ 628,    0],
       [ 660,    2],
       [ 665,    2],
       [ 697,    2],
       [ 702,